In [1]:
import logging
class FilterModule(logging.Filter):
    def __init__(self, modulename=""):
        self.modulename = modulename
    def filter(self, rec):
        return not rec.name.startswith(self.modulename)

logging.basicConfig(format='[%(asctime)s][%(levelname)s][%(name)s] %(message)s', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
for handler in logger.handlers:
    handler.addFilter(FilterModule("numba"))
    handler.addFilter(FilterModule("matplotlib"))    

In [2]:
# import logging
# logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.DEBUG)
import os
# os.environ["NUMBA_DISABLE_INTEL_SVML"] = "1"

# os.environ["NUMBA_OPT"] = "0"

import ebisim as eb
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (8,6)
plt.rcParams['figure.dpi'] = 100
# eb.simulation._advanced.compile_adv_model()
print(os.environ.get("NUMBA_DISABLE_INTEL_SVML", "Not set"))

[2020-07-07 12:32:43,189][DEBUG][ebisim.xs] Defining _normpdf.
[2020-07-07 12:32:43,191][DEBUG][ebisim.xs] Defining cxxs.
[2020-07-07 12:32:43,195][DEBUG][ebisim.xs] Defining eixs_vec.
[2020-07-07 12:32:43,197][DEBUG][ebisim.xs] Defining eixs_mat.
[2020-07-07 12:32:43,199][DEBUG][ebisim.xs] Defining rrxs_vec.
[2020-07-07 12:32:43,201][DEBUG][ebisim.xs] Defining rrxs_mat.
[2020-07-07 12:32:43,202][DEBUG][ebisim.xs] Defining drxs_vec.
[2020-07-07 12:32:43,204][DEBUG][ebisim.xs] Defining drxs_mat.
[2020-07-07 12:32:43,205][DEBUG][ebisim.xs] Defining precompute_rr_quantities.
[2020-07-07 12:32:43,207][DEBUG][ebisim.xs] Defining lookup_lotz_factors.
[2020-07-07 12:32:43,208][DEBUG][ebisim.xs] Defining eixs_energyscan.
[2020-07-07 12:32:43,209][DEBUG][ebisim.xs] Defining rrxs_energyscan.
[2020-07-07 12:32:43,211][DEBUG][ebisim.xs] Defining _eirr_e_samp.
[2020-07-07 12:32:43,212][DEBUG][ebisim.xs] Defining drxs_energyscan.
[2020-07-07 12:32:43,569][DEBUG][ebisim.utils] Defining load_dr_data.


Not set


In [4]:

k = eb.Target.get_ions("K", 1e-8 * 1.5e15, kT=80)
ne = eb.Target.get_gas("Ne", .1e-11, 5e-3)

# k = eb.Target.get_ions("K", .1*1e-8 * 1.5e15, kT_per_q=80)
# ne = eb.Target.get_gas("Ne", 100*.1e-11, 1e-4)


# k = eb.Target.get_gas("Ar", .5e-10, 1e-4)
dev = eb.Device.get(1, 8000, 1e-4, 0.8, 1300, 2, 0.005, n_grid=500,)
print(ne.n)
print(k.n)
# res = eb.advanced_simulation(dev, k,60, rates=True)

print(eb.BackgroundGas.get("Ar", 1e-10).n0/1e12)

[2020-07-07 12:32:54,910][DEBUG][ebisim.simulation._advanced] Device.get(1, 8000, 0.0001, 0.8, 1300, 2, 0.005, None, None, None, 500)
[2020-07-07 12:32:54,912][DEBUG][ebisim.simulation._advanced] Device.get: Call fd_system_nonuniform_grid.
[2020-07-07 12:32:54,912][DEBUG][ebisim.simulation._advanced] Device.get: Trap - Call boltzmann_radial_potential_linear_density_ebeam.
[2020-07-07 12:32:54,913][DEBUG][ebisim.simulation._advanced] Device.get: Barrier - Call boltzmann_radial_potential_linear_density_ebeam.
[2020-07-07 12:32:54,914][DEBUG][ebisim.simulation._advanced] Device.get: Create and return Device instance.


[7.58482099e+02 1.00000000e-10 1.00000000e-10 1.00000000e-10
 1.00000000e-10 1.00000000e-10 1.00000000e-10 1.00000000e-10
 1.00000000e-10 1.00000000e-10 1.00000000e-10]
[1.0e-10 1.5e+07 1.0e-10 1.0e-10 1.0e-10 1.0e-10 1.0e-10 1.0e-10 1.0e-10
 1.0e-10 1.0e-10 1.0e-10 1.0e-10 1.0e-10 1.0e-10 1.0e-10 1.0e-10 1.0e-10
 1.0e-10 1.0e-10]
2.41432350534664


In [7]:
TMAX = 1
T0 = 0.0001
kr, nr = eb.advanced_simulation(
    dev, [k, ne],TMAX,
    options=eb.ModelOptions(RADIAL_DYNAMICS=True),
    solver_kwargs={"dense_output":True, "max_step":1},
    rates=True
)
eb.simulation._advanced.compile_adv_model()
# kr2, nr2 = eb.advanced_simulation(
#     dev, [k, ne],TMAX,
#     options=eb.ModelOptions(RADIAL_DYNAMICS=True),
#     solver_kwargs={"dense_output":True, "max_step":1},
#     rates=True
# )
# np.allclose(kr.res.y, kr2.res.y)

[2020-07-07 12:33:03,020][INFO][ebisim.simulation._advanced] Preparing advanced simulation.
[2020-07-07 12:33:03,022][INFO][ebisim.simulation._advanced] device = Device.get(1.0, 8000.0, 0.0001, 0.8, 1300.0, 2.0, 0.005, 1651.1229895140636, 3183.098861837907, 95.97432887405196, 500).
[2020-07-07 12:33:03,023][INFO][ebisim.simulation._advanced] targets = [Target(Element: Potassium (K, Z = 19, A = 39)), Target(Element: Neon (Ne, Z = 10, A = 20))].
[2020-07-07 12:33:03,023][INFO][ebisim.simulation._advanced] t_max = 1.
[2020-07-07 12:33:03,024][INFO][ebisim.simulation._advanced] bg_gases = None.
[2020-07-07 12:33:03,024][INFO][ebisim.simulation._advanced] options = ModelOptions(EI=True, RR=True, CX=True, DR=False, SPITZER_HEATING=True, COLLISIONAL_THERMALISATION=True, ESCAPE_AXIAL=True, ESCAPE_RADIAL=True, RECOMPUTE_CROSS_SECTIONS=False, RADIAL_DYNAMICS=True, IONISATION_HEATING=True).
[2020-07-07 12:33:03,025][INFO][ebisim.simulation._advanced] rates = True.
[2020-07-07 12:33:03,025][INFO][

/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:671: RuntimeWarning: invalid value encountered in collisional_escape_rate
  R_ax_co      = plasma.collisional_escape_rate(ri, w_ax) * n
/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:688: RuntimeWarning: invalid value encountered in collisional_escape_rate
  R_ra_co      = plasma.collisional_escape_rate(ri, w_ra) * n
/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:588: RuntimeWarning: divide by zero encountered in ion_coll_rate
  q_T, self.q
/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:588: RuntimeWarning: invalid value encountered in ion_coll_rate
  q_T, self.q


 Integration: 5100 calls, t = 2.0031e-04 s

/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:616: RuntimeWarning: divide by zero encountered in true_divide
  dkT[1:]  += R_ei[:-1] / n_r[1:] * (kT[:-1] - kT[1:])
/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:617: RuntimeWarning: divide by zero encountered in true_divide
  dkT[1:]  += R_ei[:-1] / n_r[1:] * iheat[:-1]
/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:617: RuntimeWarning: invalid value encountered in add
  dkT[1:]  += R_ei[:-1] / n_r[1:] * iheat[:-1]
/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:625: RuntimeWarning: divide by zero encountered in true_divide
  dkT[:-1] += R_rr[1:] / n_r[:-1] * (kT[1:] - kT[:-1])
/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:626: RuntimeWarning: divide by zero encountered in true_divide
  dkT[:-1]  -= R_rr[1:] / n_r[:-1] * iheat[1:]
/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:648: RuntimeWarning: divide by zero encountered in true_div

 Integration: 5200 calls, t = 2.5907e-04 s

...

 Rates: 1 / 279

/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:588: RuntimeWarning: divide by zero encountered in ion_coll_rate
  q_T, self.q
/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:588: RuntimeWarning: invalid value encountered in ion_coll_rate
  q_T, self.q
/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:671: RuntimeWarning: invalid value encountered in collisional_escape_rate
  R_ax_co      = plasma.collisional_escape_rate(ri, w_ax) * n
/home/hpahl/Repos/EBISSimulation/ebisim/simulation/_advanced.py:688: RuntimeWarning: invalid value encountered in collisional_escape_rate
  R_ra_co      = plasma.collisional_escape_rate(ri, w_ra) * n


 Rates: 201 / 279

[2020-07-07 12:33:40,772][DEBUG][ebisim.simulation._advanced] Assembling result of target #0.
[2020-07-07 12:33:40,773][DEBUG][ebisim.simulation._advanced] Assembling result of target #1.
[2020-07-07 12:33:40,774][DEBUG][ebisim.simulation._advanced] Arming compilation of AdvancedModel class.


Rates finished: 279 rates
Arming compilation of AdvancedModel class.
The next calls to this class and its methods may take a while.
The compiled instance lives as long as the python interpreter.


In [ ]:
print(hash(kr.res.y.data.tobytes()))
print(nr.res.message)
nr.res.nfev, nr.res.njev, 22*nr.res.njev, 22*nr.res.njev/nr.res.nfev


In [ ]:
fig = nr.plot_rate(eb.Rate.AX_RT)
nr2.plot_rate(eb.Rate.AX_RT, ax=fig.gca())
nr.plot_rate(eb.Rate.AX_CO)
nr2.plot_rate(eb.Rate.AX_CO)

kr.plot_rate(eb.Rate.AX_RT)
kr2.plot_rate(eb.Rate.AX_RT)

_=kr.plot_rate(eb.Rate.AX_CO)
_=kr2.plot_rate(eb.Rate.AX_CO)

In [ ]:
fig = kr.plot_charge_states()
kr2.plot_charge_states()

# kr.plot_energy_density()
_=kr.plot_temperature()
_=kr2.plot_temperature()


In [ ]:
# nr.rates[eb.Rate.E_KIN_MEAN]
nr.plot_charge_states()
nr.plot_energy_density()
_=nr.plot_temperature()
nr.plot_rate(eb.Rate.E_KIN_MEAN)
nr.plot_rate(eb.Rate.E_KIN_FWHM)
_=nr.plot_rate(eb.Rate.SPITZER_HEATING)

In [ ]:
# nr.plot_rate(eb.Rate.RA_RT)
# nr.plot_rate(eb.Rate.RA_CO)
# kr.plot_rate(eb.Rate.RA_RT)
# _=kr.plot_rate(eb.Rate.RA_CO)

In [ ]:
_=kr.plot_rate(eb.Rate.F_EI)

In [ ]:
nr.plot_rate(eb.Rate.TRAPPING_PARAMETER_AXIAL, yscale="linear", )
kr.plot_rate(eb.Rate.TRAPPING_PARAMETER_AXIAL, yscale="linear", ylim=(0,20))

In [ ]:
kr.plot_rate(eb.Rate.COLLISION_RATE_SELF, yscale="log")
_ = kr.plot_rate(eb.Rate.COLLISION_RATE_TOTAL, yscale="log")

In [ ]:
nr.plot_rate(eb.Rate.COLLISION_RATE_SELF, yscale="log")
_ = nr.plot_rate(eb.Rate.COLLISION_RATE_TOTAL, yscale="log")

In [ ]:
kr.plot_radial_distribution_at_time(0.001)
kr.plot_radial_distribution_at_time(0.01)
kr.plot_radial_distribution_at_time(0.1)
kr.plot_radial_distribution_at_time(1.0)
kr.plot_radial_distribution_at_time(10.0)


In [ ]:
nr.plot_rate(eb.Rate.F_EI)
# nr.plot_radial_distribution_at_time(0.001)
nr.plot_radial_distribution_at_time(0.01)
nr.plot_radial_distribution_at_time(0.1)
nr.plot_radial_distribution_at_time(1.0)
nr.plot_radial_distribution_at_time(10.0)